# Preprocessing


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import torch
import random
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision.datasets import ImageFolder, DatasetFolder
import torchvision
import sys


def seed_libraries(SEED=42):
    # Python seeds
    random.seed(SEED)
    np.random.seed(SEED)
    # Torch seeds
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

SEED=42
seed_libraries(SEED=SEED)


# set computation device
runtime = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Computation device: {runtime}")

In [ ]:
import torchvision

data_file_name = 'WaferImages_en_8'
 

PATH_DATA = f'/content/drive/MyDrive/DViP/data/{data_file_name}.zip'
 
!unzip $PATH_DATA

In [ ]:
class ImageFolderManual(ImageFolder):
     def find_classes(self, dir):
        """
        Finds the class folders in a dataset.
        Args:
            dir (string): Root directory path.
        Returns:
            tuple: (classes, class_to_idx) where classes are relative to (dir), and class_to_idx is a dictionary.
        Ensures:
            No class is a subdirectory of another.
        """
        if sys.version_info >= (3, 5):
            # Faster and available in Python 3.5 and above
            classes = [d.name for d in os.scandir(dir) if d.is_dir() and (d.name == "circle" or d.name == "splinter")]
        else:
            classes = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]
        classes.sort()
        class_to_idx = {'circle': 0, 'splinter': 1}
        #class_to_idx = {classes[i]: i for i in range(len(classes))}
        return classes, class_to_idx

In [ ]:
image_dataset = ImageFolderManual(f'/content/{data_file_name}')
print("Imageset Length: " + str(len(image_dataset)))

In [ ]:
shuffle = True
random_state = 42

# Prepare Dataset into Train and Val and and Test (60:20:20)

dataset_size = len(image_dataset)
indices = list(range(dataset_size))

train_val_indices, test_indices = train_test_split(indices,
                                                    stratify=[image_dataset.targets[i] for i in indices],
                                                    test_size=0.20,
                                                    shuffle=shuffle,
                                                   random_state=random_state,
                                                   )

train_indices, val_indices = train_test_split(train_val_indices,
                                            stratify=[image_dataset.targets[i] for i in train_val_indices],
                                            test_size=0.25,
                                            shuffle=shuffle,
                                            random_state=random_state,
                                            )

train_subset = Subset(image_dataset, train_indices)
val_subset = Subset(image_dataset, val_indices)
test_subset = Subset(image_dataset, test_indices)


In [ ]:
print(len(train_subset))
print(len(val_subset))
print(len(test_subset))

In [ ]:
from torchvision import transforms


train_transform = transforms.Compose([
                                      transforms.Resize((256,256)),
                                      transforms.Grayscale(num_output_channels=1),
                                      transforms.ToTensor()
                                      ])

X_train = [train_transform(img[0]) for img in image_dataset]
X_train = [np.array(img) for img in X_train]


In [ ]:
X_train = np.array(X_train)
X_train = X_train.reshape(X_train.shape[0],-1)
print(X_train.shape)

# The DeepSMOTE researches saved the loaded images in a .txt file. Thus, the same approach was used.
np.savetxt('/content/splinter_and_circle_256.txt', X_train)

In [ ]:
labelset = []
for img in image_dataset:
  labelset.append(img[1])
print(labelset)
labelset = np.array(labelset)
print(labelset.shape)
np.savetxt('/content/splinter_and_circle_256_label.txt', labelset)

In [ ]:
from collections import Counter
print(Counter(labelset))

In [ ]:
# The given orginal DeepSMOTE-sourcecode is more extensive.
# I cleaned up the codeblock down to what is actually used. 
# Of course, also many things were changed to fit this project and also to reduce the complexity of this experiment.

import collections
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
import numpy as np
from sklearn.neighbors import NearestNeighbors
import time
import os

print(torch.version.cuda) #10.1
t3 = time.time()
##############################################################################
"""args for AE"""

args = {}
args['lambda'] = 0.0001       # hyper param for weight decay
args['lr'] = 0.0001           # learning rate for Adam optimizer 
args['epochs'] = 200          # how many epochs to run for
args['batch_size'] = 4        # batch size for SGD
args['train'] = True          # train networks if True, else load networks from

##############################################################################

batch_size = args['batch_size']

## create encoder model and decoder model
class Encoder(nn.Module):
    def __init__(self, args):
        super(Encoder, self).__init__()

        ###########
        ### HEXANET START
        ###########
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=7, stride=2, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(num_features=32)

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=2, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(num_features=64)

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(num_features=128)

        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn4 = nn.BatchNorm2d(num_features=256)

        self.conv5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn5 = nn.BatchNorm2d(num_features=512)
        
        self.relu = nn.LeakyReLU(negative_slope=0.2, inplace=True)

        self.fc1 = nn.Linear(in_features=512*8*8, out_features=50)
        ###########
        ### HEXANET END
        ###########
      
    def forward(self, x):
                
        ###########
        ### HEXANET START
        ###########
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.relu(self.bn4(self.conv4(x)))
        x = self.relu(self.bn5(self.conv5(x)))
        x = x.view(x.shape[0], 512*8*8)
        x = self.fc1(x)
        ###########
        ### HEXANET END
        ###########

        return x


class Decoder(nn.Module):
    def __init__(self, args):
        super(Decoder, self).__init__()

        ###########
        ### HEXANET START
        ###########
        self.fc1 = nn.Linear(in_features=50, out_features=512*8*8)
        
        self.conv5 = nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=3, stride=2, padding=1)
        self.bn5 = nn.BatchNorm2d(num_features=256)

        self.conv4 = nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=5, stride=2, padding=1)
        self.bn4 = nn.BatchNorm2d(num_features=128)


        self.conv3 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=5, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(num_features=64)
        
        self.conv2 = nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=5, stride=2, padding=1)
        self.bn2 = nn.BatchNorm2d(num_features=32)

        self.conv1 = nn.ConvTranspose2d(in_channels=32, out_channels=1, kernel_size=6, stride=2, padding=1)
        self.bn1 = nn.BatchNorm2d(num_features=1)
        self.sigmoid = nn.Sigmoid()

        self.relu = nn.ReLU6()
        
        ###########
        ### HEXANET END
        ###########


    def forward(self, x):

        ###########
        ### HEXANET START
        ###########

        x = self.fc1(x)
        x = self.relu(x)
        x = x.view(-1, 512, 8, 8)
        x = self.relu(self.bn5(self.conv5(x)))
        x = self.relu(self.bn4(self.conv4(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.bn1(self.conv1(x))
        x = self.sigmoid(x)

        ###########
        ### HEXANET END
        ###########

        return x

###############################################################################

for i in range(1):
    encoder = Encoder(args)
    decoder = Decoder(args)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    decoder = decoder.to(device)
    encoder = encoder.to(device)

    train_on_gpu = torch.cuda.is_available()

    criterion = nn.MSELoss()
    criterion = criterion.to(device)
    
    # Here one has to specify where the above saved .txt-files will be saved
    dec_x = np.loadtxt('/content/splinter_and_circle_256.txt') 
    dec_y = np.loadtxt('/content/splinter_and_circle_256_label.txt')
    dec_x = dec_x.reshape(dec_x.shape[0],1,256,256)   

    num_workers = 0

    tensor_x = torch.Tensor(dec_x)
    tensor_y = torch.tensor(dec_y,dtype=torch.long)
    mnist_bal = TensorDataset(tensor_x,tensor_y) 
    train_loader = torch.utils.data.DataLoader(mnist_bal, batch_size=batch_size,shuffle=True,num_workers=num_workers)

    best_loss = np.inf

    t0 = time.time()
    if args['train']:
        enc_optim = torch.optim.Adam(encoder.parameters(), lr = args['lr'], weight_decay = args['lambda'] )
        dec_optim = torch.optim.Adam(decoder.parameters(), lr = args['lr'], weight_decay = args['lambda'])
    
        for epoch in range(args['epochs']):
            train_loss = 0.0
            tmse_loss = 0.0
            tdiscr_loss = 0.0
            encoder.train()
            decoder.train()
        
            for images,labs in train_loader:
                
                encoder.zero_grad()
                decoder.zero_grad()
                images, labs = images.to(device), labs.to(device)
                labsn = labs.detach().cpu().numpy()
                z_hat = encoder(images)
                x_hat = decoder(z_hat)
                mse = criterion(x_hat,images)
                          
                resx = []
                resy = []
            
                tc = np.random.choice(2,1)
                
                xbeg = dec_x[dec_y == tc]
                ybeg = dec_y[dec_y == tc] 
                xlen = len(xbeg)
                nsamp = min(xlen, 100)
                ind = np.random.choice(list(range(len(xbeg))),nsamp,replace=False)
                xclass = xbeg[ind]
                yclass = ybeg[ind]
            
                xclen = len(xclass)
                xcminus = np.arange(1,xclen)
                
                xcplus = np.append(xcminus,0)
                xcnew = (xclass[[xcplus],:])
                xcnew = xcnew.reshape(xcnew.shape[1],xcnew.shape[2],xcnew.shape[3],xcnew.shape[4])
            
                xcnew = torch.Tensor(xcnew)
                xcnew = xcnew.to(device)
            
                xclass = torch.Tensor(xclass)
                xclass = xclass.to(device)
                xclass = encoder(xclass)
            
                xclass = xclass.detach().cpu().numpy()
            
                xc_enc = (xclass[[xcplus],:])
                xc_enc = np.squeeze(xc_enc)
            
                xc_enc = torch.Tensor(xc_enc)
                xc_enc = xc_enc.to(device)
                
                ximg = decoder(xc_enc)
                
                mse2 = criterion(ximg,xcnew)
            
                comb_loss = mse2 + mse
                comb_loss.backward()

                enc_optim.step()
                dec_optim.step()
            
                train_loss += comb_loss.item()*images.size(0)
                tmse_loss += mse.item()*images.size(0)
                tdiscr_loss += mse2.item()*images.size(0)
            
                 
            # print avg training statistics 
            train_loss = train_loss/len(train_loader)
            tmse_loss = tmse_loss/len(train_loader)
            tdiscr_loss = tdiscr_loss/len(train_loader)
            print('Epoch: {} \tTrain Loss: {:.6f} \tmse loss: {:.6f} \tmse2 loss: {:.6f}'.format(epoch,
                    train_loss,tmse_loss,tdiscr_loss))
            
        
        
            #store the best encoder and decoder models
            if train_loss < best_loss and train_loss < 0.1:
                print('Saving..')
                # Here one has to specify were the encoder and decoder models should be stored, as these are later used in the Generator block below
                path_enc = '/content/DeepSMOTE_encoder.pth'
                path_dec = '/content/DeepSMOTE_decoder.pth'
             
                torch.save(encoder.state_dict(), path_enc)
                torch.save(decoder.state_dict(), path_dec)
        
                best_loss = train_loss
              
    t1 = time.time()
    print('total time(min): {:.2f}'.format((t1 - t0)/60))             
 
t4 = time.time()
print('final time(min): {:.2f}'.format((t4 - t3)/60))

In [ ]:
# -*- coding: utf-8 -*-

import collections
import torch
import torch.nn as nn
import numpy as np
from sklearn.neighbors import NearestNeighbors
import os
print(torch.version.cuda) #10.1
import time
from collections import Counter
t0 = time.time()
##############################################################################
"""args for models"""

args = {}
args['lambda'] = 0.0001       # hyper param for weight decay
args['lr'] = 0.0001           # learning rate for Adam optimizer 
args['epochs'] = 200          # how many epochs to run for (actually iterate till a train loss of ~0.05 is enough)
args['batch_size'] = 4        # batch size for SGD
args['train'] = True          # train networks if True, else load networks from

##############################################################################

## create encoder model and decoder model
class Encoder(nn.Module):
    def __init__(self, args):
        super(Encoder, self).__init__()

        ###########
        ### HEXANET START
        ###########

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=7, stride=2, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(num_features=32)

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=2, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(num_features=64)

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(num_features=128)

        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn4 = nn.BatchNorm2d(num_features=256)

        self.conv5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn5 = nn.BatchNorm2d(num_features=512)
        
        self.relu = nn.LeakyReLU(negative_slope=0.2, inplace=True)

        self.fc1 = nn.Linear(in_features=512*8*8, out_features=50)

        ###########
        ### HEXANET END
        ###########
        



    def forward(self, x):
        
        ###########
        ### HEXANET START
        ###########
      
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.relu(self.bn4(self.conv4(x)))
        x = self.relu(self.bn5(self.conv5(x)))
        x = x.view(x.shape[0], 512*8*8)
        x = self.fc1(x)

        ###########
        ### HEXANET END
        ###########

        return x

class Decoder(nn.Module):
    def __init__(self, args):
        super(Decoder, self).__init__()

        ###########
        ### HEXANET START
        ###########

        self.fc1 = nn.Linear(in_features=50, out_features=512*8*8)
        
        self.conv5 = nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=3, stride=2, padding=1)
        self.bn5 = nn.BatchNorm2d(num_features=256)

        self.conv4 = nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=5, stride=2, padding=1)
        self.bn4 = nn.BatchNorm2d(num_features=128)


        self.conv3 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=5, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(num_features=64)
        
        self.conv2 = nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=5, stride=2, padding=1)
        self.bn2 = nn.BatchNorm2d(num_features=32)

        self.conv1 = nn.ConvTranspose2d(in_channels=32, out_channels=1, kernel_size=6, stride=2, padding=1)
        self.bn1 = nn.BatchNorm2d(num_features=1)
        self.sigmoid = nn.Sigmoid()

        self.relu = nn.ReLU6()
        self.pool = nn.MaxUnpool2d(kernel_size=2)
        
        ###########
        ### HEXANET END
        ###########


    def forward(self, x):

        ###########
        ### HEXANET START
        ###########

        x = self.fc1(x)
        x = self.relu(x)
        x = x.view(-1, 512, 8, 8)

        x = self.relu(self.bn5(self.conv5(x)))
        x = self.relu(self.bn4(self.conv4(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.bn1(self.conv1(x))
        x = self.sigmoid(x)


        ###########
        ### HEXANET END
        ###########

        return x
   

##############################################################################

def biased_get_class1(c):
    xbeg = dec_x[dec_y == c]
    ybeg = dec_y[dec_y == c]
    return xbeg, ybeg

def G_SM1(X, y,n_to_sample,cl):

    # n_neigh can be played around with
    n_neigh = 5 + 1
    nn = NearestNeighbors(n_neighbors=n_neigh, n_jobs=1)
    nn.fit(X)
    dist, ind = nn.kneighbors(X)

    # generating samples
    base_indices = np.random.choice(list(range(len(X))),n_to_sample)
    neighbor_indices = np.random.choice(list(range(1, n_neigh)),n_to_sample)

    X_base = X[base_indices]
    X_neighbor = X[ind[base_indices, neighbor_indices]]

    samples = X_base + np.multiply(np.random.rand(n_to_sample,1),
            X_neighbor - X_base)

    return samples, [cl]*n_to_sample

#############################################################################
np.printoptions(precision=5,suppress=True)

# DeepSMOTE normally would allow to use cross-validation during training and therefore uses 5 different files
# For simplicity, we do not, and therefore hardcode the patch in this one-iteration loop
for m in range(1):
    dec_x = np.loadtxt('/content/drive/MyDrive/DViP/data/splinter_and_circle_256.txt') 
    dec_y = np.loadtxt('/content/drive/MyDrive/DViP/data/splinter_and_circle_256_label.txt')

    print('train imgs before reshape ',dec_x.shape) 
    print('train labels ',dec_y.shape)

    dec_x = dec_x.reshape(dec_x.shape[0],1,256,256)

    print('train imgs after reshape ',dec_x.shape) #(45000,3,32,32)
    
    
    train_on_gpu = torch.cuda.is_available()
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    path_enc = '/content/DeepSMOTE_encoder.pth'
    path_dec = '/content/DeepSMOTE_decoder.pth'

    encoder = Encoder(args)
    encoder.load_state_dict(torch.load(path_enc), strict=False)
    encoder = encoder.to(device)

    decoder = Decoder(args)
    decoder.load_state_dict(torch.load(path_dec), strict=False)
    decoder = decoder.to(device)

    encoder.eval()
    decoder.eval()

    # Normally DeepSMOTE would calculate the imbalance between the classes automatically.
    # As we achieved better results if the models are just trained on the class(es) that should be generated, we only use respective classes here.
    dataset_distribution = Counter(dec_y)
    #imbal = Counter(dec_y)
    #imbal = [1096, 79]


    resx = []
    resy = [] 

    for i in range(0, len(dataset_distribution)):
        xclass, yclass = biased_get_class1(i) 
        xclass = torch.Tensor(xclass)
        xclass = xclass.to(device)
        xclass = encoder(xclass)

        xclass = xclass.detach().cpu().numpy()

        # The standard DeepSMOTE code calculates the imbalance between classes by:
        #n = dataset_distribution[max(dataset_distribution, key=dataset_distribution.get)] - dataset_distribution[i]
        # We hardcoded "658" here, because in the train set there are 658 "good" images, so we want to generate the missing ones to balance the imbalance.
        n = 658 - dataset_distribution[i]
        print("Generate ", n, " new images for class", i)
        print(len(xclass), len(yclass))
        xsamp, ysamp = G_SM1(xclass,yclass,n,i)

        ysamp = np.array(ysamp)
    
        xsamp = torch.Tensor(xsamp)
        xsamp = xsamp.to(device)
        ximg = decoder(xsamp)

        ximn = ximg.detach().cpu().numpy()
        resx.append(ximn)
        resy.append(ysamp)

    resx1 = np.vstack(resx)
    resy1 = np.hstack(resy)
    resx1 = resx1.reshape(resx1.shape[0],-1)
    
    # Normally, DeepSMOTE outputs the original images 'dec_x1' and the generated ones 'resx1'.
    # This way, the output normally is a balanced image set.
    # As we did not generate images for all classes and just for circle and splinter, we only want to output the generated ones seperately.
    # Therefore, below we save "resx1" and "resy1" in the .txt-file instead of "combx" and "comby"
    #dec_x1 = dec_x.reshape(dec_x.shape[0],-1)
    #combx = np.vstack((resx1,dec_x1))
    #comby = np.hstack((resy1,dec_y))


    ifile = '/content/DeepSMOTE_dataset.txt'
    np.savetxt(ifile, resx1)
    
    lfile = '/content/DeepSMOTE_dataset_labels.txt'
    np.savetxt(lfile,resy1) 
    print()

t1 = time.time()
print('final time(min): {:.2f}'.format((t1 - t0)/60))

In [ ]:
import numpy as np

gen_imgs = np.loadtxt('/content/DeepSMOTE_dataset.txt')
gen_imgs = gen_imgs.reshape(gen_imgs.shape[0],1,256,256)
gen_labels = np.loadtxt('/content/DeepSMOTE_dataset_labels.txt')
print(Counter(gen_labels))

In [ ]:
import numpy as np

gen_imgs = resx1.reshape(resx1.shape[0],1,256,256)
gen_labels = resy1
print(Counter(gen_labels))

In [ ]:
# This codeblock is only used for debugging purposes to actually check how the generated images look.

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

def show_img(img_array, i):
  img_array = img_array.reshape(256,256)
  img = Image.fromarray(np.uint8(img_array * 255) , 'L')
  # Uncomment the line below to save pictures in the specified path
  #img.save('/content/drive/MyDrive/SampleGeneration/DeepSMOTE/Generated Images/circle/' + str(i) +'.png')
  plt.imshow(img, interpolation='nearest', cmap='gray')
  plt.show()
for i in range(0, 1000):
    show_img(gen_imgs[i].astype(np.uint8), i)
    print(resy1[i])